# Introduction

## LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic

# llm = OpenAI(): deprecatetd
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?"): deprecated
b = chat.predict("How many planets are there?")

b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

## Predict Messages

In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian.",
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

## Prompt Templates

In [4]:
from langchain.prompts import PromptTemplate


template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [5]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

## OutputParser and LCEL

In [6]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [7]:
template =  ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?",
)

result = chat.predict_messages(prompt)

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [8]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?",
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

## Chaining Chains

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [10]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on makin traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- 2 tsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1/2 tsp turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, paprika, turmeric, salt, and pepper. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.
2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and pl

AIMessageChunk(content="To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a suitable alternative. One popular substitute for chicken in Indian dishes is paneer, which is a type of Indian cottage cheese. Here's how you can modify the recipe:\n\n**Vegetarian Chicken Tikka Masala**\n\n**Ingredients:**\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) tomato sauce\n- 1 cup heavy cream\n- 2 tsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp paprika\n- 1/2 tsp turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n\n**Instructions:**\n1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, paprika, turmeric, salt, and pepper. Add the paneer pieces and marinate for at least 1 hour, preferably

# Model IO

## FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# t = PromptTemplate(
#     template = "What is the capital of {country}?",
#     input_variables = ["country"],
# )

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "What do you know about Spain?",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

chain = prompt | chat

chain.invoke({
    "country": "Germany",
})

AI:
        Here is what I know:
        Capital: Berlin
        Population: 83 million
        Language: German

AIMessageChunk(content='AI:\n        Here is what I know:\n        Capital: Berlin\n        Population: 83 million\n        Language: German')

## FewShotChatMessagePromptTemplate

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"),
    ("ai", "{answer}"),
])

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "Spain",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "Italy",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert."),
    example_prompt,
    ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({
    "country": "Germany",
})


        Here is what I know:
        Capital: Berlin
        Population: 83 million
        

AIMessageChunk(content='\n        Here is what I know:\n        Capital: Berlin\n        Population: 83 million\n        ')

In [24]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "What do you know about Spain?",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 300,
)

random_example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = random_example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

prompt.format(country="France")

'Human:What do you know about Italy?\nAI:\n        Here is what I know:\n        Capital: Rome\n        Population: 60 million\n        \n\nHuman: What do you know about France?'

## Serialization and Composition

In [25]:
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.json")

prompt.format(country="France")

'What is the capital of France?'

In [26]:
prompt = load_prompt("prompt.yaml")

prompt.format(country="France")

'What is the capital of France?'

In [32]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts = prompts,
)

full_prompt.format(
    character="Pirate",
    example_question="What is the meaning of life?",
    example_answer="The meaning of life is to find the treasure!",
    question="How old are you?",
)

chain = full_prompt | chat

chain.invoke({
    "character": "Pirate",
    "example_question": "What is the meaning of life?",
    "example_answer": "The meaning of life is to find the treasure!",
    "question": "How old are you?",
})

Arrr, me age be as old as the seas themselves! But a true pirate never reveals his exact age, matey.

AIMessageChunk(content='Arrr, me age be as old as the seas themselves! But a true pirate never reveals his exact age, matey.')

## Caching

In [35]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(temperature=0.1)

chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans and smooth the tops with a spatula.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted i

In [36]:
chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and end

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans and smooth the tops with a spatula.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted i

In [38]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="cache.db"))

chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.02s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted into the center of the cakes comes o

## Serialization

In [39]:
from langchain.callbacks import get_openai_callback

set_llm_cache(None)

with get_openai_callback() as usage:
    chat.predict("How do you make a cake?")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

In [40]:
from langchain.llms.openai import OpenAI

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")

chat.save("model.json")

In [41]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/Users/davidminjae/.local/share/virtualenvs/fullstack-gpt-Wpua4Cas/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/davidminjae/.local/share/virtualenvs/fullstack-gpt-Wpua4Cas/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})